In [13]:
# Overview: Linear Regression on Ray Dalio's Bubble/Depression Gauge
#Train model using data from 15 historical deflationary debt cycles over the last 150 years.
#Data received was directly extracted from the book, "Principles for Navigating Big Debt Crises"

# Libraries
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

#Features being used to describe the Bubble/Debt predictor
FACTOR_COLS = [
    "CoreInflation","ChangeDebt/GDP","SRi","Money0/GDP",
    "R.GDPg","Curve","Equityg","R.FXxTWI"
]

#Train Linear Regression setting features as X values to describe the "BDG" column (Y)
df = pd.read_csv("Database2.csv").dropna(subset=FACTOR_COLS + ["BDG"])
X = df[FACTOR_COLS].values
y = df["BDG"].values

m = LinearRegression().fit(X, y)
print("R^2 (in-sample):", r2_score(y, m.predict(X)))

# quick coef print
for f,c in zip(FACTOR_COLS, m.coef_):
    print(f"{f:15s} {c:+.6f}")
print("Intercept        ", f"{m.intercept_:+.6f}")

# --- predict ---
pred = pd.read_csv("PredictionBDG.csv")
pred["BDG_pred_linear"] = m.predict(pred[FACTOR_COLS].values)
pred.to_csv("Prediction2_linear_predictions.csv", index=False)
print("\nSaved to Prediction2_linear_predictions.csv")
print(pred[["BDG_pred_linear"]].to_string(index=False))


R^2 (in-sample): 0.2825066548659295
CoreInflation   +0.072166
ChangeDebt/GDP  +0.014547
SRi             +0.028700
Money0/GDP      +0.010038
R.GDPg          +0.115400
Curve           +0.050855
Equityg         +0.366656
R.FXxTWI        +0.010932
Intercept         -0.796057

Saved to Prediction2_linear_predictions.csv
 BDG_pred_linear
       -0.099378
       -0.018390
       -0.023966
       -0.402010
        0.660342
        0.260311
        0.301925
        0.324302
        0.047494


In [14]:
#Overview: Cubic Spline Interpolation on Ray Dalio's Bubble/Depression Gauge
#Train model using data from 15 historical deflationary debt cycles over the last 150 years.
#Data to train the model received was directly extracted from the book, "Principles for Navigating Big Debt Crises"
#Model is then placed on US data from 2015-2025 to see what the current Bubble/depression Gauge is

import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, SplineTransformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score

F = ["CoreInflation","ChangeDebt/GDP","SRi","Money0/GDP","R.GDPg","Curve","Equityg","R.FXxTWI"]

# train
df = pd.read_csv("Database2.csv").dropna(subset=F+["BDG"])
X, y = df[F], df["BDG"]

# natural cubic splines (linear extrapolation), per feature
spline = ColumnTransformer(
    [(f, SplineTransformer(degree=3, n_knots=9, extrapolation="linear"), [i])
     for i,f in enumerate(F)],
    remainder="drop"
)

pipe = make_pipeline(
    StandardScaler(with_mean=False),  # spline outputs are already centered-ish
    spline,
    Ridge(alpha=1.0, random_state=42)
).fit(X, y)

print("R^2 (in-sample, splines+ridge):", r2_score(y, pipe.predict(X)))

# predict
pred = pd.read_csv("PredictionBDG.csv")
pred["BDG_pred_splines_ridge"] = pipe.predict(pred[F])
pred.to_csv("Prediction2_splines_ridge_predictions.csv", index=False)



R^2 (in-sample, splines+ridge): 0.609084201792315
